# Exercise 1: Data cleaning

Before doing actual data analysis, we usually first need to clean the data. 
This might involve steps such as dealing with missing values and encoding categorical variables as integers.
In this exercise, you will perform such steps based on the Titanic passenger data.

1. Load the Titanic data set in `titanic.csv` located in the `data/` folder.
2. Report the number of observations with missing `Age`, for example using [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.isna.html).
3. Compute the average age in the data set. Use the following approaches and compare your results:
    1.  Use pandas's [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) method.
    2.  Convert the `Age` column to a NumPy array using [`to_numpy()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html). Experiment with NumPy's [`np.mean()`](https://numpy.org/doc/2.0/reference/generated/numpy.mean.html) and [`np.nanmean()`](https://numpy.org/doc/2.0/reference/generated/numpy.nanmean.html) to see if you obtain the same results.
4. Replace the all missing ages with the mean age you computed above, rounded to the nearest integer.
   Note that in "real" applications, replacing missing values with sample means is usually not a good idea.
5. Convert this updated `Age` column to integer type using [`astype()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html).
6. Generate a new column `Female` which takes on the value one if `Sex` is equal to `"female"` and zero otherwise. 
   This is called an _indicator_ or _dummy_ variable, and is preferrable to storing such categorical data as strings.
   Delete the original column `Sex`.
7. Save your cleaned data set as `titanic-clean.csv` using [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) with `,` as the field separator.
   Tell `to_csv()` to *not* write the `DataFrame` index to the CSV file as it's not needed in this example.

In [2]:
import pandas as pd
import numpy as np

DATA_PATH = '../../data' #go up two folders, and go to data folder
filename = f'{DATA_PATH}/titanic.csv'

# Load sample data set of Titanic passengers. Individual fields are separated using a comma, which is the default.
df = pd.read_csv(filename, sep=',')

#way 1: foolish way
missing = 0
for i in df["Age"].isna():
    missing += i
print(missing)


177


In [3]:
#way 2: smarter
df.info() #prints the non-null count

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        204 non-null    object 
 9   Embarked     889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [4]:
#finding the mean
avg_age = df["Age"].mean()
print(f"Avrage age is {round(avg_age,2)}")

Avrage age is 29.7


In [5]:
#finding the mean with numpy
np_array_age = df["Age"].to_numpy()
np.mean(np_array_age) #return nan, since some of the values are nan
print(f"Avrage age is {round(np.nanmean(np_array_age),2)}")

Avrage age is 29.7


In [6]:
#replace the nan-values with the mean #never loop through columns in a data frame

df["Age"].fillna(avg_age) #replaces all na-values with avg_age

# or
df.loc[df["Age"].isna(), "Age"] = avg_age  #replace
df["Age"]


0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [7]:
df["Female2"] = 0
for i, value in enumerate(df["Sex"]):
    df.loc[i,"Female2"] = int(value == "female")
    

In [8]:
#Convert this updated Age column to integer type using astype()
df["Age"] = df["Age"].astype(int)

In [9]:
df["female"] = (df["Sex"] == "female").astype(int) #making male to 0 and female to 1

df[["female","Sex"]] #checks the dataset

#delete the sex-column
#df.drop(columns=["Sex"])

#alternative
del df["Sex"]
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Age          891 non-null    int64  
 5   Ticket       891 non-null    object 
 6   Fare         891 non-null    float64
 7   Cabin        204 non-null    object 
 8   Embarked     889 non-null    object 
 9   Female2      891 non-null    int64  
 10  female       891 non-null    int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 76.7+ KB


In [10]:
#store modified dataframe
df.to_csv(f"{DATA_PATH}/titanic-cleaned.csv")

***
# Exercise 2: Selecting subsets of data

In this exercise, you are asked to select subsets of macroeconomic data for the United States based on some criteria.

1.  Load the annual data from FRED which are located in `FRED_annual.xlsx` 
    in the `data/FRED` folder.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `Year` as the DataFrame index.

4.  Print all observations for the 1960s decade using at least two different methods.

5.  Using the data in the column `GDP`, compute the annual GDP growth in percent and store it in the column `GDP_growth`. Select the years in which

    1.  GDP growth was above 5%.
    2.  GDP growth was negative, but inflation as still above 5% (such episodes are called "stagflation" since usually negative GDP growth is associated with low inflation).
        
        Use at least two methods to select such years.

    *Hint:* You can compute changes relative to the previous observation using the 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) method.


In [11]:
filename = f"{DATA_PATH}/FRED/FRED_annual.xlsx"

df = pd.read_excel(filename)

df.info() #overview of the columns
df.columns.to_list() #other way of doing it, more for programming

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       70 non-null     int64  
 1   GDP        70 non-null     float64
 2   CPI        70 non-null     float64
 3   UNRATE     70 non-null     float64
 4   FEDFUNDS   70 non-null     float64
 5   INFLATION  69 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 3.4 KB


['Year', 'GDP', 'CPI', 'UNRATE', 'FEDFUNDS', 'INFLATION']

In [12]:
df = df.set_index("Year") #saves it permanently
df

,GDP,CPI,UNRATE,FEDFUNDS,INFLATION
Year,,,,,
1954,2877.7,26.9,5.6,1.0,NaN
1955,3083.0,26.8,4.4,1.8,-0.371747
1956,3148.8,27.2,4.1,2.7,1.492537
1957,3215.1,28.1,4.3,3.1,3.308824
1958,3191.2,28.9,6.8,1.6,2.846975
...,...,...,...,...,...
2019,20715.7,255.7,3.7,2.2,1.831939
2020,20267.6,258.8,8.1,0.4,1.212358
2021,21494.8,271.0,5.4,0.1,4.714065


In [13]:
#print all observations for 1960

#alt 1
print(df.query("Year >= 1960 & Year <= 1969"))

#alt 2
df.loc[1960:1970] #the last item is included. 

# alt 3
df.iloc[6:16] #not as convinient have to know the 

         GDP   CPI  UNRATE  FEDFUNDS  INFLATION
Year                                           
1960  3500.3  29.6     5.5       3.2   1.369863
1961  3590.1  29.9     6.7       2.0   1.013514
1962  3810.1  30.3     5.6       2.7   1.337793
1963  3976.1  30.6     5.6       3.2   0.990099
1964  4205.3  31.0     5.2       3.5   1.307190
1965  4478.6  31.5     4.5       4.1   1.612903
1966  4773.9  32.5     3.8       5.1   3.174603
1967  4904.9  33.4     3.8       4.2   2.769231
1968  5145.9  34.8     3.6       5.7   4.191617
1969  5306.6  36.7     3.5       8.2   5.459770


,GDP,CPI,UNRATE,FEDFUNDS,INFLATION
Year,,,,,
1960,3500.3,29.6,5.5,3.2,1.369863
1961,3590.1,29.9,6.7,2.0,1.013514
1962,3810.1,30.3,5.6,2.7,1.337793
1963,3976.1,30.6,5.6,3.2,0.990099
1964,4205.3,31.0,5.2,3.5,1.307190
1965,4478.6,31.5,4.5,4.1,1.612903
1966,4773.9,32.5,3.8,5.1,3.174603
1967,4904.9,33.4,3.8,4.2,2.769231
1968,5145.9,34.8,3.6,5.7,4.191617


***
# Exercise 3: Labor market statistics for the US

In this exercise, you are asked to compute some descriptive statistics for the unemployment rate 
and the labor force participation (the fraction of the working-age population in the labor force, i.e., individuals who are either employed or unemployed) for the United States.

1.  Load the monthly time series from FRED which are located in `FRED_monthly.csv` 
    in the `data/FRED` folder.

    *Hint:* You can use `pd.read_csv(..., parse_dates=['DATE'])` to automatically
    parse strings stored in the `DATE` column as dates.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `DATE` as the DataFrame index. Using the date index, select all observations from the first three months
    of the year 2020.

4.  For the columns `UNRATE` (unemployment rate) and `LFPART` (labor force participation), compute and report 
    the mean, minimum and maximum values for the whole sample. Round your results 
    to one decimal digit.

    *Hint:* You can use the DataFrame methods 
    [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html),
    [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html), and 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html)
    to compute the desired statistics.

    *Hint:* You can use the DataFrame method
    [`round()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html)
    to truncate the number of decimal digits.

5.  You are interested in how the average unemployment rate evolved over the last 
    few decades. 

    -   Add a new column `Decade` to the DataFrame which contains the 
        starting year for each decade (e.g., this value should be 1950
        for the years 1950-1959, and so on).

        *Hint:* The decade can be computed from the column `Year` using 
        truncated integer division:
        ```python
        df['Year'] // 10 * 10
        ```

    -   Write a loop to compute and report the average unemployment rate (column `UNRATE`) 
        for each decade.

        Include only the decades from 1950 to 2010 for which you have all
        observations. 

In [ ]:
import pandas as pd
filename = "../../data/FRED/FRED_monthly.csv"

df = pd.read_csv(filename)
df.info() #list of columns and the number of non-missing observations
df = df.set_index("DATE")

#select all observations from the first tree months of 2020
df.loc["2020-01-01":"2020-03-01"]

#For Unrate and LFPART compute mean, min, max values for whole samlpe, and round
print(f"{round(df["UNRATE"].mean(),1)} is the mean of unrate")
print(f"{df["UNRATE"].min()} is the min of unrate")
print(f"{df["UNRATE"].max()} is the max of unrate")

#evolvement of UNRATE last decades
df["Decade"] = df["Year"]//10*10 #new decade column

for i in range(1950,2010,10):
    start = f"{i}-01-01"
    end = f"{i+9}-12-31"
    decade_data = df.loc[start:end]
    mean_unrate = decade_data["UNRATE"].mean()
    print(f"in the {i} the mean was: {mean_unrate}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      924 non-null    object 
 1   Year      924 non-null    int64  
 2   Month     924 non-null    int64  
 3   CPI       924 non-null    float64
 4   UNRATE    924 non-null    float64
 5   FEDFUNDS  846 non-null    float64
 6   REALRATE  516 non-null    float64
 7   LFPART    924 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 57.9+ KB
5.7 is the mean of unrate
2.5 is the min of unrate
14.8 is the max of unrate
in the 1950 the mean was: 4.511666666666667
in the 1960 the mean was: 4.779166666666667
in the 1970 the mean was: 6.2175
in the 1980 the mean was: 7.2725
in the 1990 the mean was: 5.762500000000001
in the 2000 the mean was: 5.541666666666667


***
# Exercise 4: Working with string data (advanced)

Most of the data we deal with contain strings, i.e., text data (names, addresses, etc.). Often, such data is not in the format needed for analysis, and we have to perform additional string manipulation to extract the exact data we need. This can be achieved using the pandas [string methods](https://pandas.pydata.org/docs/user_guide/text.html#string-methods).

To illustrate, we use the Titanic data set for this exercise.

1.  Load the Titanic data and restrict the sample to men. (This simplifies the task. Women in this data set have much more complicated names as they contain both their husband's and their maiden name)
2.  Print the first five observations of the `Name` column. As you can see, the data is stored in the format _"Last name, Title First name"_ where title is something like Mr., Rev., etc.
3. Split the `Name` column by `,` to extract the last name and the remainder as separate columns. You can achieve this using the [`partition()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.partition.html#pandas.Series.str.partition) string method.
4. Split the remainder (containing the title and first name) using the space character `" "` as separator to obtain individual columns for the title and the first name.
5. Store the three data series in the original `DataFrame` (using the column names `FirstName`, `LastName` and `Title`) and delete the `Name` column which is no longer needed.
6. Finally, extract the ship deck from the values in `Cabin`. The ship deck is the first character in the string stored in `Cabin` (A, B, C, ...). You extract the first character using the 
[`get()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get.html#pandas.Series.str.get) string method. Store the result in the column `Deck`.

*Hint*: Pandas's string methods can be accessed using the `.str` attribute. For example, to partition values in the column `Name`, you need to use
```python
df['Name'].str.partition()
```


In [1]:
#chat GPT Code
# Exercise 4 — Working with string data (advanced)
# ------------------------------------------------
# What this script does (step-by-step, matching the task):
# 1) Load the Titanic data and keep only men.
# 2) Print the first five values of the Name column.
# 3) Split Name by "," to get LastName and the remainder.
# 4) Split the remainder by a space to get Title and FirstName.
# 5) Store FirstName, LastName, Title in the same DataFrame and drop Name.
# 6) Extract Deck from Cabin using the first character (string .get()).

import pandas as pd

# --- 1) LOAD DATA -------------------------------------------------------------
# If your CSV is in a file, set the path here (e.g., "titanic.csv").
# The inline dataset you pasted can be saved to a file and read from that path.
csv_path = "../../data/titanic.csv"
df = pd.read_csv(csv_path)

# Keep only men. Copy to avoid SettingWithCopy warnings later.
df = df[df["Sex"] == "male"].copy()

# --- 2) PEEK AT THE NAME COLUMN ----------------------------------------------
# Shows the first 5 raw names (verifies the format we’re about to parse).
print("First 5 names (men only):")
print(df["Name"].head(), end="\n\n")

# --- 3) SPLIT 'Name' INTO 'LastName' + remainder using partition(",") --------
# .str.partition(",") returns a DataFrame with three parts:
# [0] text before the separator, [1] the separator itself, [2] text after.
# We use it because names are of the form: "Last name, Title First name"
name_parts = df["Name"].str.partition(",")
# [0] -> Last name (strip spaces); [2] -> "Title First name" (strip spaces)
df["LastName"] = name_parts[0].str.strip()
remainder = name_parts[2].str.strip()

# --- 4) SPLIT remainder INTO 'Title' and 'FirstName' -------------------------
# The remainder looks like "Mr. Owen Harris" (or "Master Gosta Leonard", etc.)
# We split on the first two spaces only (n=2) so extra middle names stay intact in part[2].
# We only need the first two tokens: Title and FirstName.
parts = remainder.str.split(" ", n=2, expand=True)
# parts[0] -> Title (e.g., "Mr.", "Master", "Don.", "Dr.", "Rev.", "Col.", "Capt.", etc.)
# parts[1] -> FirstName (first given name token; middle names—if any—are ignored by design here)
df["Title"] = parts[0].str.strip()
df["FirstName"] = parts[1].str.strip()

# Note: Some rows can have missing age/ticket/cabin etc.; names for men are consistently parseable
# with this approach on the standard Titanic training set.

# --- 5) KEEP the three new columns; DROP 'Name' and any temporary helpers ----
df.drop(columns=["Name"], inplace=True)

# --- 6) MAKE 'Deck' from the first character of 'Cabin' ----------------------
# Deck is the first character in the Cabin code (A, B, C, ...). For NaN cabins, result stays NaN.
df["Deck"] = df["Cabin"].astype(str).str.get(0)
# Fix "n" from "nan" string for missing cabins (produced by astype(str)); set those back to NaN.
df.loc[df["Cabin"].isna(), "Deck"] = pd.NA

# --- OPTIONAL: show the result ------------------------------------------------
print("Result preview (men only) with FirstName, LastName, Title, Deck:")
print(df[["FirstName", "LastName", "Title", "Deck"]].head())

# If you want to save the processed data:
# df.to_csv("titanic_processed_men.csv", index=False)


First 5 names (men only):
0          Braund, Mr. Owen Harris
4         Allen, Mr. William Henry
5                 Moran, Mr. James
6          McCarthy, Mr. Timothy J
7    Palsson, Master Gosta Leonard
Name: Name, dtype: object

Result preview (men only) with FirstName, LastName, Title, Deck:
  FirstName  LastName   Title  Deck
0      Owen    Braund     Mr.  <NA>
4   William     Allen     Mr.  <NA>
5     James     Moran     Mr.  <NA>
6   Timothy  McCarthy     Mr.     E
7     Gosta   Palsson  Master  <NA>


In [3]:
df

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,LastName,Title,FirstName,Deck
0,1,0,3,male,22.0,A/5 21171,7.2500,NaN,S,Braund,Mr.,Owen,<NA>
4,5,0,3,male,35.0,373450,8.0500,NaN,S,Allen,Mr.,William,<NA>
5,6,0,3,male,NaN,330877,8.4583,NaN,Q,Moran,Mr.,James,<NA>
6,7,0,1,male,54.0,17463,51.8625,E46,S,McCarthy,Mr.,Timothy,E
7,8,0,3,male,2.0,349909,21.0750,NaN,S,Palsson,Master,Gosta,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,884,0,2,male,28.0,C.A./SOTON 34068,10.5000,NaN,S,Banfield,Mr.,Frederick,<NA>
884,885,0,3,male,25.0,SOTON/OQ 392076,7.0500,NaN,S,Sutehall,Mr.,Henry,<NA>
886,887,0,2,male,27.0,211536,13.0000,NaN,S,Montvila,Rev.,Juozas,<NA>
889,890,1,1,male,26.0,111369,30.0000,C148,C,Behr,Mr.,Karl,C
